<a href="https://colab.research.google.com/github/SSAFY11-Data-ML-Study/20240417_Tangerine_Data/blob/main/20240417_%EA%B9%80%EB%AF%BC%ED%98%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

data: https://dacon.io/competitions/official/236176/overview/description

목차   
0. 데이터     
1. 모델      
    1.1 Catboost    
    1.2 LightGBM    
2. 제출     
3. 딥러닝 기반 모델(작업중)    

# 0. Data

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [32]:
train = pd.read_csv("/content/drive/MyDrive/data_0417/train.csv")
int_trade = pd.read_csv("/content/drive/MyDrive/data_0417/international_trade.csv")
test = pd.read_csv("/content/drive/MyDrive/data_0417/test.csv")

In [7]:
train.isnull().sum()

ID             0
timestamp      0
item           0
corporation    0
location       0
supply(kg)     0
price(원/kg)    0
dtype: int64

In [8]:
int_trade.isnull().sum()

기간       0
품목명      0
수출 중량    0
수출 금액    0
수입 중량    0
수입 금액    0
무역수지     0
dtype: int64

In [9]:
test.isnull().sum()

ID             0
timestamp      0
item           0
corporation    0
location       0
dtype: int64

In [15]:
train.keys()

Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply(kg)',
       'price(원/kg)'],
      dtype='object')

In [12]:
int_trade.keys()

Index(['기간', '품목명', '수출 중량', '수출 금액', '수입 중량', '수입 금액', '무역수지'], dtype='object')

In [16]:
test.keys()

Index(['ID', 'timestamp', 'item', 'corporation', 'location'], dtype='object')

In [24]:
train.head(10)

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
5,TG_A_J_20190106,2019-01-06,TG,A,J,0.0,0.0
6,TG_A_J_20190107,2019-01-07,TG,A,J,44995.0,1474.0
7,TG_A_J_20190108,2019-01-08,TG,A,J,26975.0,1326.0
8,TG_A_J_20190109,2019-01-09,TG,A,J,29265.0,1428.0
9,TG_A_J_20190110,2019-01-10,TG,A,J,21226.0,1433.0


In [25]:
int_trade.head(20)

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990
1,2019-01,양파,821330,222,4003206,1118,-896
2,2019-01,쪽파,60,1,93405,128,-127
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
4,2019-01,방울다다기 양배추,0,0,7580,38,-38
5,2019-01,양배추,184650,94,395802,90,4
6,2019-01,결구(結球) 상추,85200,95,258096,165,-70
7,2019-01,위트루프 치커리(Witloof chicory)[시코리엄 인티부스 변종 포리오섬(Ci...,0,0,90,1,-1
8,2019-01,당근,23150,22,7466150,2955,-2934
9,2019-01,오이류(신선한 것이나 냉장한 것으로 한정한다),10340,28,0,0,28


In [22]:
train.size

415779

In [23]:
int_trade.size

8918

In [57]:
from sklearn.preprocessing import LabelEncoder

In [116]:
train = pd.read_csv("/content/drive/MyDrive/data_0417/train.csv")
test = pd.read_csv("/content/drive/MyDrive/data_0417/test.csv")
def preprocess(data):
    data.drop("ID",axis = 1,inplace = True)
    data['timestamp'] = pd.to_datetime(data['timestamp'],format='%Y-%m-%d')
    data['year'] = data['timestamp'].dt.year
    data['month'] = data['timestamp'].dt.month
    data['day'] = data['timestamp'].dt.day
    data['day_of_week'] = data['timestamp'].dt.dayofweek
    data['holiday'] = data.apply(lambda x:0 if x['day_of_week']<5 else 1,axis = 1)
    data.drop(columns=["timestamp"],axis=1,inplace=True)
    data.rename(columns={'supply(kg)':'supply', 'price(원/kg)':'price'},inplace=True)
    if 'supply' in data.keys():
        data.drop(columns=["supply"],axis=1,inplace=True)
    for i in ['item','corporation','location']:
        le = LabelEncoder()
        le = le.fit(data[i])
        data[i] = le.transform(data[i])
    return data
preprocess(train)
preprocess(test)

,item,corporation,location,year,month,day,day_of_week,holiday
0,4,0,0,2023,3,4,5,1
1,4,0,0,2023,3,5,6,1
2,4,0,0,2023,3,6,0,0
3,4,0,0,2023,3,7,1,0
4,4,0,0,2023,3,8,2,0
...,...,...,...,...,...,...,...,...
1087,3,5,0,2023,3,27,0,0
1088,3,5,0,2023,3,28,1,0
1089,3,5,0,2023,3,29,2,0
1090,3,5,0,2023,3,30,3,0


In [73]:
train

,item,corporation,location,price,year,month,day,day_of_week,holiday
0,4,0,0,0.0,2019,1,1,1,0
1,4,0,0,0.0,2019,1,2,2,0
2,4,0,0,1728.0,2019,1,3,3,0
3,4,0,0,1408.0,2019,1,4,4,0
4,4,0,0,1250.0,2019,1,5,5,1
...,...,...,...,...,...,...,...,...,...
59392,3,5,0,468.0,2023,2,27,0,0
59393,3,5,0,531.0,2023,2,28,1,0
59394,3,5,0,574.0,2023,3,1,2,0
59395,3,5,0,523.0,2023,3,2,3,0


In [74]:
test

,item,corporation,location,year,month,day,day_of_week,holiday
0,4,0,0,2023,3,4,5,1
1,4,0,0,2023,3,5,6,1
2,4,0,0,2023,3,6,0,0
3,4,0,0,2023,3,7,1,0
4,4,0,0,2023,3,8,2,0
...,...,...,...,...,...,...,...,...
1087,3,5,0,2023,3,27,0,0
1088,3,5,0,2023,3,28,1,0
1089,3,5,0,2023,3,29,2,0
1090,3,5,0,2023,3,30,3,0


In [76]:
from sklearn.model_selection import train_test_split

In [117]:
def split(train):
    target = train['price']
    train = train.drop('price',axis = 1)
    x_train, x_valid, y_train, y_valid = train_test_split(train,target,test_size = 0.2)
    return x_train, x_valid, y_train, y_valid
x_train, x_valid, y_train, y_valid = split(train)

# 1. Model

## 1.1 Catboost

In [75]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 6.2 MB/s eta 0:00:00


In [79]:
from catboost import CatBoostRegressor,Pool

In [81]:
train_pool = Pool(data=x_train,label=y_train)
valid_pool = Pool(data=x_valid,label=y_valid)

In [ ]:
cat = CatBoostRegressor(iterations=2000, learning_rate = 0.1, random_strength = 0.5, depth = 7, random_state = 42)
cat.fit(train_pool,eval_set=(valid_pool),verbose = 100)

In [84]:
test_pool = Pool(data = test)
test_pred = cat.predict(test_pool)

In [87]:
for i in range(len(test_pred)):
    if test_pred[i]<0:
        test_pred[i] = 0

## 1.2 LightGBM

In [112]:
import lightgbm as lgb

In [118]:
train_lgb = lgb.Dataset(x_train,label = y_train)
test_lgb= lgb.Dataset(x_valid, label=y_valid)

params = {
    'objective' : 'regression',
    'metric': 'rmse',
    'num_leaves':30,
    'lr':0.1,
    'feature_fraction':0.9
}

model = lgb.train(params,train_lgb,num_boost_round=100)

[LightGBM] [Warning] Unknown parameter: lr
[LightGBM] [Warning] Unknown parameter: lr
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014140 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 73
[LightGBM] [Info] Number of data points in the train set: 47517, number of used features: 8
[LightGBM] [Info] Start training from score 1119.328240


In [120]:
test_pred = model.predict(test)

In [124]:
for i in range(len(test_pred)):
    if test_pred[i]<0:
        test_pred[i] = 0

# 2. Submit

In [127]:
submit = pd.read_csv("/content/drive/MyDrive/data_0417/sample_submission.csv")
submit['answer'] = test_pred
submit.to_csv("/content/drive/MyDrive/data_0417/trial2_2.csv",index=False)


In [126]:
submit

,ID,answer
0,TG_A_J_20230304,3368.254298
1,TG_A_J_20230305,34.515418
2,TG_A_J_20230306,3728.121302
3,TG_A_J_20230307,3770.022834
4,TG_A_J_20230308,3770.022834
...,...,...
1087,RD_F_J_20230327,531.822816
1088,RD_F_J_20230328,531.822816
1089,RD_F_J_20230329,536.273699
1090,RD_F_J_20230330,510.532644


In [100]:
train.value_counts('corporation')

corporation
0    13707
4    13707
3    12184
2     9138
1     7615
5     3046
dtype: int64

In [101]:
test.value_counts('corporation')

corporation
0    252
4    252
3    224
2    168
1    140
5     56
dtype: int64

In [102]:
test

,item,corporation,location,year,month,day,day_of_week,holiday
0,4,0,0,2023,3,4,5,1
1,4,0,0,2023,3,5,6,1
2,4,0,0,2023,3,6,0,0
3,4,0,0,2023,3,7,1,0
4,4,0,0,2023,3,8,2,0
...,...,...,...,...,...,...,...,...
1087,3,5,0,2023,3,27,0,0
1088,3,5,0,2023,3,28,1,0
1089,3,5,0,2023,3,29,2,0
1090,3,5,0,2023,3,30,3,0


# 3. 딥러닝 모델 (LSTM)

In [ ]:
##작업중(LSTM)

In [106]:
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 19.4 MB/s eta 0:00:00


In [108]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
## config

In [111]:
train.value_counts('item')

item
4    15230
0    13707
3    12184
2    10661
1     7615
dtype: int64